## Máquina de pintado - Modelos de Simulación
#### - Rivera Juan Matías

In [1]:
import datetime

##### Función transforma un datetime en un string con valor igual al tiempo(time)

In [2]:
def transformarTiempo(tiempo):
    st = datetime.datetime.strptime(str(tiempo), "%Y-%m-%d %H:%M:%S")
    return st.time()

#### Clase de cola

In [3]:
class Cola:
    def __init__(self,q,TiempoLlegada,DeltaLlegada,Limite):
        self.q=q
        self.tiempoL=TiempoLlegada
        self.deltaL = DeltaLlegada
        self.contador = 0
        self.limite = Limite
        
    def Llegada(self,PS):
        tiempoA = self.tiempoL
        if PS.estado == 0:
            PS.estado = 1
            PS.tiempoFS = self.tiempoL + PS.deltaFS[PS.contador]
            PS.contador+=1
        else:
            self.q += 1

        self.tiempoL += self.deltaL[self.contador]
        self.contador +=1
        return tiempoA
    
    def ReinicioContador(self):
        if self.contador > len(self.deltaL)-1:
            self.contador = 0
            

#### Clase de Puesto de Servicio

In [4]:
class PuestoDeServicio:
    def __init__(self,Disposicion,Estado,TiempoFS,DeltaFS,TiempoInh,TiempoHab,DeltaDesc,DeltaInh,Limite):
        self.disposicion = Disposicion
        self.estado = Estado
        self.tiempoFS = TiempoFS
        self.deltaFS = DeltaFS
        self.contador = 0
        self.limite = Limite
        self.tiempoInh = TiempoInh
        self.tiempoHab = TiempoHab
        self.deltaDesc = DeltaDesc
        self.deltaInh = DeltaInh
        
    def FinDeServicio(self,Cola):
        tiempoA = self.tiempoFS
        
        if Cola.q > 0:
            Cola.q -= 1
            self.tiempoFS += self.deltaFS[self.contador]
            self.contador += 1
        else:
            self.estado = 0
            self.tiempoFS = self.limite
            
        return tiempoA
        
    def ReinicioContador(self):
        if self.contador > len(self.deltaFS)-1:
            self.contador = 0

            
    def Inhabilitar(self):
        tiempoA = self.tiempoInh
        self.tiempoFS += deltaDesc
        self.tiempoHab = tiempoA + deltaDesc
        self.disposicion = 0
        self.tiempoInh = self.limite
        return tiempoA       
        
    def Habilitar(self):
        tiempoA = self.tiempoHab
        self.tiempoInh = tiempoA + self.deltaInh
        self.tiempoHab = self.limite
        self.disposicion = 1
        return tiempoA


##### Función para imprimir Tiempo, recibe variables en el siguiente orden: Hora Actual, LLegada del Cliente, Fin de Servicio, N° Cola (q) y Estado del Puesto de Servicio(PS)'

In [12]:
def imprimirTiempo(Cola,PS,TiempoA):
    
    tiempoActual = transformarTiempo(TiempoA)
    tiempoLlegada = transformarTiempo(Cola.tiempoL)
    tiempoFinServicio = transformarTiempo(PS.tiempoFS)
    tiempoInhabilitar = transformarTiempo(PS.tiempoInh)
    tiempoHabilitar = transformarTiempo(PS.tiempoHab)
    if PS.estado==0:
        tiempoFinServicio = "        "
        
    if PS.disposicion==0:
        tiempoInhabilitar = "        "
    else:
        tiempoHabilitar = "        "

    print(tiempoActual," | ",tiempoLlegada," | ",tiempoFinServicio," | ",tiempoInhabilitar," | ",tiempoHabilitar," | ",Cola.q," | ",PS.estado," | ",PS.disposicion)

##### Función vector de deltas de tiempo por segundo'

In [13]:
def delta(tiemposDelta):
    listaDelta = []
    for i in tiemposDelta:
        listaDelta.append(datetime.timedelta(seconds = i))
    return listaDelta

##### Función asignación de tiempo'

In [14]:
def tiempo(horas,minutos,segundos):
    tiempo = datetime.datetime.combine(datetime.datetime.today(),datetime.time(horas,minutos,segundos,0)) 
    return tiempo

In [15]:
def timedelta(i):
    return datetime.timedelta(seconds = i)

##### Declaración de los tiempos iniciales y los deltas

In [18]:
#Instancia del limite(HORA DE SERVICIO)'
limite = tiempo(12,0,0)

#Instancia de los tiempos iniciales
tiempoActual = tiempo(10,30,0)
tiempoL = tiempo(10,30,20)
tiempoFS = limite + timedelta(30)
tiempoInh = tiempo(10,31,34)
tiempoHab = limite + timedelta(30)




#Tiempos delta particulares del práctico
tiempoDL = [65,6,2,21,42]
tiempoDFS = [5,10]


deltaL = delta(tiempoDL)
deltaFS = delta(tiempoDFS)
deltaDesc = timedelta(60)
deltaInh = timedelta(30)




#Instancia de la Cola y del Puesto de Servicio
cola = Cola(0,tiempoL,deltaL,limite+timedelta(30))
PS = PuestoDeServicio(1,0,tiempoFS,deltaFS,tiempoInh,tiempoHab,deltaDesc,deltaInh,limite+timedelta(30))


##### Programa principal, un while que imprime toda la matriz.

In [19]:
print("  Actual  |  LLegada   | Fin de S.  |  Descanso  |  Vuelta T. |  q  |  PS |  S |")
print("--------------------------------------------------------------------------------")

#Se imprime primero el estado inicial
imprimirTiempo(cola,PS,tiempoActual)


while tiempoActual < limite:
        #Si el tiempo de llegada de cliente es menor al fin de servicio'
        if cola.tiempoL <= PS.tiempoFS  and cola.tiempoL < limite and cola.tiempoL <= PS.tiempoInh:
            
            tiempoActual = cola.Llegada(PS)  
            imprimirTiempo(cola,PS,tiempoActual)
            
        #Si el tiempo de fin de servicio es menor que el de llegada'
        elif PS.tiempoFS <= cola.tiempoL and PS.tiempoFS < PS.tiempoInh and PS.tiempoFS < limite and PS.estado == 1:
            tiempoActual = PS.FinDeServicio(cola)
        
            imprimirTiempo(cola,PS,tiempoActual)
         
        
        elif PS.tiempoInh <= PS.tiempoFS and PS.tiempoInh <= cola.tiempoL:
            tiempoActual = PS.Inhabilitar()
            imprimirTiempo(cola,PS,tiempoActual)
            
            while cola.tiempoL <= PS.tiempoHab:
                tiempoActual = cola.Llegada(PS)
                imprimirTiempo(cola,PS,tiempoActual)
                cola.ReinicioContador()
                PS.ReinicioContador()
            tiempoActual = PS.Habilitar()
            imprimirTiempo(cola,PS,tiempoActual)
            
        #Cierre
        else:
            if PS.estado == 1:
                print("El último cliente en el PS saldrá a las horas:",transformarTiempo(PS.tiempoFS))
            break
        
        #Reinicio de contadores
        cola.ReinicioContador()
        PS.ReinicioContador()

print("Clientes sin atención:",cola.q)
    

  Actual  |  LLegada   | Fin de S.  |  Descanso  |  Vuelta T. |  q  |  PS |  S |
--------------------------------------------------------------------------------
10:30:00  |  10:30:20  |            |  10:31:34  |            |  0  |  0  |  1
10:30:20  |  10:31:25  |  10:30:25  |  10:31:34  |            |  0  |  1  |  1
10:30:25  |  10:31:25  |            |  10:31:34  |            |  0  |  0  |  1
10:31:25  |  10:31:31  |  10:31:35  |  10:31:34  |            |  0  |  1  |  1
10:31:31  |  10:31:33  |  10:31:35  |  10:31:34  |            |  1  |  1  |  1
10:31:33  |  10:31:54  |  10:31:35  |  10:31:34  |            |  2  |  1  |  1
10:31:34  |  10:31:54  |  10:32:35  |            |  10:32:34  |  2  |  1  |  0
10:31:54  |  10:32:36  |  10:32:35  |            |  10:32:34  |  3  |  1  |  0
10:32:34  |  10:32:36  |  10:32:35  |  10:33:04  |            |  3  |  1  |  1
10:32:35  |  10:32:36  |  10:32:40  |  10:33:04  |            |  2  |  1  |  1
10:32:36  |  10:33:41  |  10:32:40  |  10:33:04 

10:55:04  |  10:55:16  |  10:54:15  |  10:55:34  |            |  3  |  1  |  1
10:54:15  |  10:55:16  |  10:54:20  |  10:55:34  |            |  2  |  1  |  1
10:54:20  |  10:55:16  |  10:54:30  |  10:55:34  |            |  1  |  1  |  1
10:54:30  |  10:55:16  |  10:54:35  |  10:55:34  |            |  0  |  1  |  1
10:54:35  |  10:55:16  |            |  10:55:34  |            |  0  |  0  |  1
10:55:16  |  10:56:21  |  10:55:26  |  10:55:34  |            |  0  |  1  |  1
10:55:26  |  10:56:21  |            |  10:55:34  |            |  0  |  0  |  1
10:55:34  |  10:56:21  |            |            |  10:56:34  |  0  |  0  |  0
10:56:21  |  10:56:27  |  10:56:26  |            |  10:56:34  |  0  |  1  |  0
10:56:27  |  10:56:29  |  10:56:26  |            |  10:56:34  |  1  |  1  |  0
10:56:29  |  10:56:50  |  10:56:26  |            |  10:56:34  |  2  |  1  |  0
10:56:34  |  10:56:50  |  10:56:26  |  10:57:04  |            |  2  |  1  |  1
10:56:26  |  10:56:50  |  10:56:36  |  10:57:04  |  

11:17:15  |  11:17:56  |            |  11:18:04  |            |  0  |  0  |  1
11:17:56  |  11:19:01  |  11:18:06  |  11:18:04  |            |  0  |  1  |  1
11:18:04  |  11:19:01  |  11:19:06  |            |  11:19:04  |  0  |  1  |  0
11:19:01  |  11:19:07  |  11:19:06  |            |  11:19:04  |  1  |  1  |  0
11:19:04  |  11:19:07  |  11:19:06  |  11:19:34  |            |  1  |  1  |  1
11:19:06  |  11:19:07  |  11:19:11  |  11:19:34  |            |  0  |  1  |  1
11:19:07  |  11:19:09  |  11:19:11  |  11:19:34  |            |  1  |  1  |  1
11:19:09  |  11:19:30  |  11:19:11  |  11:19:34  |            |  2  |  1  |  1
11:19:11  |  11:19:30  |  11:19:21  |  11:19:34  |            |  1  |  1  |  1
11:19:21  |  11:19:30  |  11:19:26  |  11:19:34  |            |  0  |  1  |  1
11:19:26  |  11:19:30  |            |  11:19:34  |            |  0  |  0  |  1
11:19:30  |  11:20:12  |  11:19:40  |  11:19:34  |            |  0  |  1  |  1
11:19:34  |  11:20:12  |  11:20:40  |            |  

11:41:34  |  11:41:41  |  11:40:46  |  11:42:04  |            |  0  |  1  |  1
11:40:46  |  11:41:41  |            |  11:42:04  |            |  0  |  0  |  1
11:41:41  |  11:41:47  |  11:41:46  |  11:42:04  |            |  0  |  1  |  1
11:41:46  |  11:41:47  |            |  11:42:04  |            |  0  |  0  |  1
11:41:47  |  11:41:49  |  11:41:57  |  11:42:04  |            |  0  |  1  |  1
11:41:49  |  11:42:10  |  11:41:57  |  11:42:04  |            |  1  |  1  |  1
11:41:57  |  11:42:10  |  11:42:02  |  11:42:04  |            |  0  |  1  |  1
11:42:02  |  11:42:10  |            |  11:42:04  |            |  0  |  0  |  1
11:42:04  |  11:42:10  |            |            |  11:43:04  |  0  |  0  |  0
11:42:10  |  11:42:52  |  11:42:20  |            |  11:43:04  |  0  |  1  |  0
11:42:52  |  11:43:57  |  11:42:20  |            |  11:43:04  |  1  |  1  |  0
11:43:04  |  11:43:57  |  11:42:20  |  11:43:34  |            |  1  |  1  |  1
11:42:20  |  11:43:57  |  11:42:25  |  11:43:34  |  